---
### **Convolution caculate Parameters**  
---
> #### 합성곱 연산에 따른 파라미터 계산
>> - **Convolution Parameter**  
>>> **1. Input Data Height : H**  
>>> **2. Input Data Width : W**  
>>> **3. Filter Height: FH**  
>>> **4. Filter Width : WD**  
>>> **5. Stride : S**  
>>> **6. Padding : P**    


![Convolution](/home/park/coding/study/DL/DeepLearning/Convolution/Convolution/outputsize.gif)

![formula](/home/park/coding/study/DL/DeepLearning/Convolution/Convolution/formula.png)

---
> **Convolution Parameter Fomula** 
>> **InputChannel x KernelWidth x KernelHeight x OutputChannel + Bias(Filter)**   
>>> **(Output Channel은 Filter의 값을 의미한다.)**  

 

---
> **BatchNormalization Parameter Configuration**   
>> **1. Gamma: Scaling Parameter**  
>> **2. Beta: Shift Parameter**  
>> **3. Mean: Non-Trainable Params**  
>> **4. Standard deviation: Non-Trainable Params**  


 

---
### **Example Calculate Conv Param on AlexNet**

![alexnetarchi](/home/park/coding/study/DL/DeepLearning/Convolution/Convolution/alexnet_architecture.PNG)

---
>### **AlexNet Architecture**
>>- #### **Input: 227x227x3 크기의 컬러 이미지.**  
>>> **1.Conv-1: 11x11 크기의 커널 96개, stride=4, padding=0**   
>>> **2.MaxPool-1: stride 2, 3x3 max pooling layer**  
>>> **3.Conv-2: 5x5 크기의 커널 256개, stride=1, padding=2**  
>>> **4.MaxPool-2: stride 2, 3x3 max pooling layer**  
>>> **5.Conv-3: 3x3 크기의 커널 384개, stride=1, padding=1**  
>>> **6.Conv-4: 3x3 크기의 커널 384개, stride=1, padding=1**  
>>> **7.Conv-5: 3x3 크기의 커널 256개, stride=1, padding=1**  
>>> **8.Maxpool-3: stride 2, 3x3 max pooling layer**  
>>> **9.FC-1: 4096개의 fully connected layer**  
>>> **10.FC-2: 4096개의 fully connected layer**  
>>> **11.FC-3: 1000개의 fully connected layer** 

---
>### **AlexNet Architecture Code**


---
##### Import Lib

In [4]:
import torch
import torch.nn as nn
import numpy as np
from torchsummary import summary

---
##### Class Define
###### 논문에 W, H의 SIZE가 224로 잘못 표기 되어있어 227로 수정하여 사용한다.  
###### 파라미터의 계산 편의를 위하여 Relu와 LRN, Dropout은 제외하여 설계하였다. (전체버전은 Github에 업로드)

In [7]:
class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.Layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=0, bias=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, padding=2, bias=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, padding=1,bias=True),
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, padding=1,bias=True),
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1,bias=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.FC = nn.Sequential(
            nn.Linear(in_features= 9216, out_features= 4096),
            nn.Linear(in_features= 4096, out_features= 4096),
            nn.Linear(in_features= 4096, out_features= 1000),            
        )
        
        self.Init_Bias()
    
    def Init_Bias(self):
        for layer in self.Layer:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std= 0.01)
                nn.init.constant_(layer.bias, 0)
        nn.init.constant_(self.Layer[2].bias, 1)
        nn.init.constant_(self.Layer[4].bias, 1)
        nn.init.constant_(self.Layer[5].bias, 1)
    
    def forward(self, x):
        x = self.Layer(x)
        x = x.view(-1,256 * 6 *6)
        out = self.FC(x)
        return out

---
##### Main

In [27]:
device = torch.device('cpu')

if __name__ == "__main__":
    model1 = AlexNet()
    summary(model=model1, input_size= (3,227,227), device= device.type)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
         MaxPool2d-2           [-1, 96, 27, 27]               0
            Conv2d-3          [-1, 256, 27, 27]         614,656
         MaxPool2d-4          [-1, 256, 13, 13]               0
            Conv2d-5          [-1, 384, 13, 13]         885,120
            Conv2d-6          [-1, 384, 13, 13]       1,327,488
            Conv2d-7          [-1, 256, 13, 13]         884,992
         MaxPool2d-8            [-1, 256, 6, 6]               0
            Linear-9                 [-1, 4096]      37,752,832
           Linear-10                 [-1, 4096]      16,781,312
           Linear-11                 [-1, 1000]       4,097,000
Total params: 62,378,344
Trainable params: 62,378,344
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.59
Fo

---
##### Result
![result1](ConvParam\result.PNG)

---
>### **Calculate Number of Weights, Bias**
>>**Weight = Size of Kernels x Size of Kernels x Number of Channels x Number of Kernel**  
>>**Bias = Number of Channels**

In [29]:
conv1_weights = 11 * 11 * 3 * 96
conv1_bias = 96
conv1_sum = conv1_weights + conv1_bias

conv2_weights = 5 * 5 * 96 * 256
conv2_bias = 256
conv2_sum = conv2_weights + conv2_bias

conv3_weights = 3 * 3 * 256 * 384
conv3_bias = 384
conv3_sum = conv3_weights + conv3_bias

conv4_weights = 3 * 3 * 384 * 384
conv4_bias = 384
conv4_sum = conv4_weights + conv4_bias

conv5_weights = 3 * 3 * 384 * 256
conv5_bias = 256
conv5_sum = conv5_weights + conv5_bias


print("conv1_weights: ", conv1_weights)
print("conv1_bias: ", conv1_bias)
print("conv1_sum: ", conv1_sum)

print("conv2_weights: ", conv2_weights)
print("conv2_bias: ", conv2_bias)
print("conv2_sum: ", conv2_sum)

print("conv3_weights: ", conv3_weights)
print("conv3_bias: ", conv3_bias)
print("conv3_sum: ", conv3_sum)

print("conv4_weights: ", conv4_weights)
print("conv4_bias: ", conv4_bias)
print("conv4_sum: ", conv4_sum)

print("conv5_weights: ", conv5_weights)
print("conv5_bias: ", conv5_bias)
print("conv5_sum: ", conv5_sum)

  
        
    

conv1_weights:  34848
conv1_bias:  96
conv1_sum:  34944
conv2_weights:  614400
conv2_bias:  256
conv2_sum:  614656
conv3_weights:  884736
conv3_bias:  384
conv3_sum:  885120
conv4_weights:  1327104
conv4_bias:  384
conv4_sum:  1327488
conv5_weights:  884736
conv5_bias:  256
conv5_sum:  884992


---
##### Result
![result2](ConvParam\result2.PNG)